# 2. Evolve only, time dependent.

## Problem Parameters
* sigma = 1.0
* b=as_vector([2.0, 3.0])
* mu = 10**(-5)
* Using same problem parameters for all cases in this book.

## Exact Solution (steady)
* c = 16.0 
* h = x*(1-x)*y*(1-y)
* g = 2*mu**(-0.5)*(0.25^2 - (x - 0.5)^2 - (y - 0.5)^2 )
 * iliescu = c*h*(0.5+sym.atan(g)/sym.pi)

nx = [25, 50, 100, 200, 400, 800];


### Case: no time dependence solution (solution from book 1)
* dt = 0.01, T = 0.01: checking the first time step to see if solution converges

L2 = [0.368993791938, 0.365904801733,0.364453616144,0.364351562791,0.364348676342, 0.364350346201];

H1= [10.6678477777,11.3773983927,7.53380792097,4.60927008799,4.25925064503, 4.24412160496];


* dt = 0.01, T = 0.02: checking the first 2 time steps to see if solution converges

L2 = [0.368993791938, 0.365904801733, 0.364453616144,0.364351562791,0.364348676342,0.364350346201]

H1 = [10.6678477777, 11.3773983927, 7.53380792097, 4.60927008799, 4.25925064503, 4.24412160496]

Notes:
* Errors are not converging properly.

## Exact Solution (time dep)
* c = 16.0*sym.sin(sym.pi*t)
* h = x*(1-x)*y*(1-y)
* g = 2*mu**(-0.5)*(0.25**2 - (x - 0.5)**2 - (y - 0.5)**2 )
 * iliescu = c*h*(0.5+sym.atan(g)/sym.pi)

nx = [25, 50, 100, 200, 400, 800];


### Case: time dependent solution
* dt = 0.01, T = 0.01: checking the first time step to see if solution converges

L2 = [0.0239802243278,0.0239513898403,0.0237699754044,0.0237746935108,0.0237756035489,0.0237755998797];

H1 = [0.61076385581,0.749544902358,0.568590010782,0.446254919906,0.435115732696,0.434766469574];

Notes:
* Found some errors in code:
 * I forgot to time step the exact solution for good comparison.
 * Forgot to update previous time step after solution is found.
* Time stepping is definitely wrong.

### Case: time dependent solution
* dt = 1, T = 0.01, remove while loop!

L2 = [0.266828434956,0.247211208008,0.207463064338,0.152916237119,0.134728904097, 0.133319784975];

H1 = [24.5316095308,42.4136319021,62.474211178,68.8441342061,67.1809048283, 67.6577187892];

Notes:
* Convergence rate of errors are not behaving properly. Try something smart.

In [1]:
%matplotlib inline 
import matplotlib
from dolfin import *
from __future__ import print_function
import math as m
import sympy as sym
import csv

x, y, t = sym.symbols('x[0], x[1], t')
sigma = 1.0
mu = 10**(-5) 
b=as_vector([2.0, 3.0])

# Iliescu Exact Solution
c = 16.0*sym.sin(sym.pi*t)
h = x*(1-x)*y*(1-y)
g = 2*mu**(-0.5)*(0.25**2 - (x - 0.5)**2 - (y - 0.5)**2 )
ue = c*h*(0.5+sym.atan(g)/sym.pi)

# ------------------------------------------ #

ue = sym.simplify(ue)
u_code = sym.printing.ccode(ue)
u_code = u_code.replace('M_PI','DOLFIN_PI')

# du/dt - mu*Laplace(u) + div(bu) + sigma*u = f
fe = sym.diff(ue,t)
fe += - mu*(sym.diff(sym.diff(ue,x),x) + sym.diff(sym.diff(ue,y),y))
fe += b[0]*sym.diff(ue,x) + b[1]*sym.diff(ue,y)
fe += sigma*ue

f_code = sym.printing.ccode(fe)
f_code = f_code.replace('M_PI','DOLFIN_PI')

# print('u_code = ' + u_code + '\n')
# print('f_code = ' + f_code)

def compute_errors(u_e, u, t, mesh):
	L2n = errornorm(u_e, u, norm_type='L2', degree_rise=3, mesh=mesh)
	H1n = errornorm(u_e, u, norm_type='H1', degree_rise=3, mesh=mesh)
	errors = {'L2 norm': L2n, 'H1 norm': H1n}
	return 'L2, ' + str(L2n) +', H1, '+ str(H1n) +', t, '+ str(t) +'\n'

In [8]:
def evolve_t(nx, dt, T, u_code, f_code, sigma, mu, velocity):
    degree = 2
    
    t = 0.0
    u_exact = Expression(u_code, degree = degree, t = t)
    f = Expression(f_code, degree = degree, t = t)

    mesh = UnitSquareMesh(nx,nx)
    Q = FunctionSpace(mesh, "CG", degree)

    # Set up boundary condition
    u_D = Expression(u_exact.cppcode, degree = degree, t = t)
    
    def boundary(x, on_boundary):
        return on_boundary

    # Test and trial functions
    u, v = TrialFunction(Q), TestFunction(Q)
    #u_n = Function(Q)
    u_n = interpolate(u_D, Q)
    u_ = Function(Q)

    # Galerkin variational problem
    F = (u - u_n)*v*dx
    F += dt*(mu*dot(grad(v), grad(u))*dx + v*dot(velocity, grad(u))*dx + sigma*v*u*dx - f*v*dx)

    # Create bilinear and linear forms
    a1 = lhs(F)
    L = rhs(F)

    # Assemble matrices
    A1 = assemble(a1)

    # Create progress bar
    progress = Progress('Time-stepping')
    set_log_level(PROGRESS)
    
    # Outputting files
    out_file_uexact = File("time_dep/evolve_u_exact_"+str(nx)+".pvd") 
    out_file_ubar = File("time_dep/"+str(mu)+"evolve_u_"+str(nx)+".pvd") 
    ue = interpolate(u_exact, Q)
    
    # Save t = 0.0
    out_file_uexact << (ue, float(t))
    out_file_ubar << (u_, float(t))

    while t - T < DOLFIN_EPS:
        # Step 1 Solve on Coarse Grid
        t += dt
        
        u_D.t = t
        f.t = t
        u_exact.t = t
        
       
        b = assemble(L)
        bc = DirichletBC(Q, u_D, boundary)
        bc.apply(A1)
        bc.apply(b)
        
        solve(A1, u_.vector(), b, 'gmres')
        progress.update(t / T)

        out_file_uexact << (ue, float(t))
        out_file_ubar << (u_, float(t))
        
        u_n.assign(u_)

    errors = compute_errors(u_exact, u_, t, mesh)
    print(errors)

After fixing the boundary condition such that it moves forward.

t = 0.01

L2 = [0.00184782410145,0.0010581291447,0.000276852607002,4.6630251329e-05,5.6213014013e-06, 3.90191336512e-06];

H1 = [0.312363909881,0.332888253109,0.195881350549,0.0568651311196,0.0124115252922,0.00438917806714];

t = 0.02

L2 = [0.00477790918901,0.00273003989336,0.000694943973744,0.000114531225914,1.59420248785e-05,1.33224617959e-05];

H1 = [0.768480358603,0.833713592743, 0.478676790862,0.132601138766, 0.0258954719342,0.00885887604912];




In [5]:
dt=0.01
T=0.01
for nx in [25, 50, 100]:
    print('nx = '+str(nx))
    evolve_t(nx, dt, T, u_code, f_code, sigma, mu, b)

for nx in [200, 400]:
    print('nx = '+str(nx))
    evolve_t(nx, dt, T, u_code, f_code, sigma, mu, b)

nx = 25
L2, 0.00184782410145, H1, 0.312363909881, t, 0.01

L2, 0.00477790918901, H1, 0.768480358603, t, 0.02

nx = 50
L2, 0.0010581291447, H1, 0.332888253109, t, 0.01

L2, 0.00273003989336, H1, 0.833713592743, t, 0.02

nx = 100
L2, 0.000276852607002, H1, 0.195881350549, t, 0.01

L2, 0.000694943973744, H1, 0.478676790862, t, 0.02

nx = 200
L2, 4.6630251329e-05, H1, 0.0568651311196, t, 0.01

L2, 0.000114531225914, H1, 0.132601138766, t, 0.02

nx = 400
L2, 5.6213014013e-06, H1, 0.0124115252922, t, 0.01

L2, 1.59420248785e-05, H1, 0.0258954719342, t, 0.02



In [6]:
nx = 800
print('nx = '+str(nx))
evolve_t(nx, dt, T, u_code, f_code, sigma, mu, b)

nx = 800
L2, 3.90191336512e-06, H1, 0.00438917806714, t, 0.01

L2, 1.33224617959e-05, H1, 0.00885887604912, t, 0.02



With the new corrections to the code, I will run the code to T=0.5, to make sure the convergence is fine for longer periods of run time.

** Task 2 completed! Evolve only, time dependent code is fixed. 4/12/18 8:47pm **

dt = 0.01

T = 0.5


L2 = [0.37754817099,0.19194848289,0.03698847582,0.00584696741022,0.00223471532047,0.00221318872969];

H1 = [52.1983435974,48.3205959032,18.9217034051,4.39133912945,1.1871295158,1.12508797379];

In [9]:
dt = 0.01
T = 0.5
for nx in [25, 50, 100]:
    print('nx = '+str(nx))
    evolve_t(nx, dt, T, u_code, f_code, sigma, mu, b)

for nx in [200, 400]:
    print('nx = '+str(nx))
    evolve_t(nx, dt, T, u_code, f_code, sigma, mu, b)
    
nx = 800
print('nx = '+str(nx))
evolve_t(nx, dt, T, u_code, f_code, sigma, mu, b)

nx = 25
L2, 0.37754817099, H1, 52.1983435974, t, 0.51

nx = 50
L2, 0.19194848289, H1, 48.3205959032, t, 0.51

nx = 100
L2, 0.03698847582, H1, 18.9217034051, t, 0.51

nx = 200
L2, 0.00584696741022, H1, 4.39133912945, t, 0.51

nx = 400
L2, 0.00223471532047, H1, 1.1871295158, t, 0.51

nx = 800
L2, 0.00221318872969, H1, 1.12508797379, t, 0.51

